In [4]:
#imports
import tensorflow as tf                                               # For building and training the CNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For loading and augmenting images
from tensorflow.keras import layers, models                           # For defining the CNN model
import matplotlib.pyplot as plt                                       # For plotting accuracy/loss
! pip install -q kaggle
# from google.colab import files
# files.upload()

In [5]:

# STEP 2: Set up Kaggle API access
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# STEP 3: Download the MRL dataset
!kaggle datasets download -d prasadvpatil/mrl-dataset
!unzip -q mrl-dataset.zip


cp: kaggle.json: No such file or directory
chmod: /Users/michaelhamaty/.kaggle/kaggle.json: No such file or directory
Traceback (most recent call last):
  File "/Users/michaelhamaty/anaconda3/bin/kaggle", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/michaelhamaty/anaconda3/lib/python3.11/site-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michaelhamaty/anaconda3/lib/python3.11/site-packages/kaggle/api/kaggle_api_extended.py", line 1734, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michaelhamaty/anaconda3/lib/python3.11/site-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open mrl-dataset.zip, mrl-dataset.zip.zip or mrl-dataset.zip.ZIP.


In [ ]:
import os
os.listdir()
# Replace with actual folder name from previous step
dataset_path = "train"
print("Inside", dataset_path, ":", os.listdir(dataset_path))
subfolder = os.listdir(dataset_path)[0]
print("Inside", subfolder, ":", os.listdir(os.path.join(dataset_path, subfolder)))



In [ ]:
import os
print(os.listdir())


In [ ]:
import os

for root, dirs, files in os.walk("mrlEyes_2018_01"):
    for file in files:
        if file.endswith(".png"):
            print(f"Sample File: {file}")
            print(f"Full Path: {os.path.join(root, file)}")
            break
    break  # Only do the first folder


In [ ]:
import os
import cv2
import numpy as np

X = []
y = []
errors = 0

data_dir = "train"  # this should now be correct

# Walk through all subdirectories and files
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".png"):
            try:
                # Label is usually in filename: starts with 0_ or 1_
                label = int(file.split('_')[0])
                path = os.path.join(root, file)

                img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Failed to load {path}")
                    errors += 1
                    continue

                # Crop and resize (simulate head tilt region)
                h, w = img.shape
                center_x, center_y = w//2, h//2
                size = min(h, w) // 2
                cropped = img[center_y-size:center_y+size, center_x-size:center_x+size]
                cropped = cv2.resize(cropped, (64, 64))

                X.append(cropped)
                y.append(label)
            except Exception as e:
                print(f"Error loading {file}: {e}")
                errors += 1

print(f"\n✅ Loaded {len(X)} images with {errors} errors.")



In [ ]:
import os
import cv2
import numpy as np

X = []
y = []
errors = 0

dataset_path = "train"

for label_folder in os.listdir(dataset_path):
    full_path = os.path.join(dataset_path, label_folder)
    if not os.path.isdir(full_path):
        continue

    for file in os.listdir(full_path):
        if file.endswith(".png"):
            try:
                # Label from folder name
                label = 1 if label_folder.lower() == "closed_eyes" else 0

                # Load and preprocess
                img_path = os.path.join(full_path, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue

                # Crop center region to simulate head tilt
                h, w = img.shape
                center_x, center_y = w//2, h//2
                size = min(h, w) // 2
                cropped = img[center_y-size:center_y+size, center_x-size:center_x+size]
                cropped = cv2.resize(cropped, (64, 64))

                X.append(cropped)
                y.append(label)
            except Exception as e:
                print(f"Error loading {file}: {e}")
                errors += 1

print(f"✅ Loaded {len(X)} images | ❌ {errors} errors")



In [ ]:
# Normalize and prepare for CNN
X = np.array(X).astype("float32") / 255.0
X = np.expand_dims(X, axis=-1)
y = np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])



In [ ]:
from tensorflow.keras import layers, models

# Simple CNN
model = models.Sequential([
    layers.Input(shape=(64, 64, 1)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title("Head Tilt Proxy CNN (from MRL)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("mrl_head_tilt_model.h5")

# Prediction function
def predict_head_tilt(frame, model):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    h, w = gray.shape
    center_x, center_y = w // 2, h // 2
    size = min(h, w) // 2
    cropped = gray[center_y - size:center_y + size, center_x - size:center_x + size]

    try:
        resized = cv2.resize(cropped, (64, 64))
    except:
        return "UNKNOWN"  # In case the crop fails

    normalized = resized / 255.0
    input_img = np.expand_dims(normalized, axis=(0, -1))  # (1, 64, 64, 1)

    prediction = model.predict(input_img)[0][0]
    return "DROWSY" if prediction > 0.5 else "ALERT"
